In [1]:
import re
import tensorflow.keras as Keras

Tokenizer =  Keras.preprocessing.text.Tokenizer

filename = 'drive/My Drive/depression.txt'

with open(filename, encoding='utf-8-sig', errors='ignore') as f:
  text = f.read()

In [2]:
seq_length = 20
start_story = '|'*seq_length

In [3]:
text = text.lower()
text = start_story + text
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

In [4]:
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
token_list = tokenizer.texts_to_sequences([text])[0]

In [5]:
import numpy as np
np_utils = Keras.utils

In [6]:
def generate_sequences(token_list, step):
  x = []
  y = []
  for i in range(0, len(token_list)-seq_length, step):
    x.append(token_list[i: i+seq_length])
    y.append(token_list[i+seq_length])

  y = np_utils.to_categorical(y, num_classes=total_words)

  num_seq = len(x)

  print('Number of sequneces: ', num_seq, '\n')
  return x ,y, num_seq

In [7]:
step = 1
seq_length = 20
x, y, num_seq = generate_sequences(token_list, step)

x_train = np.array(x)
y = np.array(y)

Number of sequneces:  56204 



In [8]:
Dense = Keras.layers.Dense
LSTM = Keras.layers.LSTM
Input = Keras.Input
Embedding = Keras.layers.Embedding
Dropout = Keras.layers.Dropout
Model = Keras.models.Model

In [9]:
n_units = 256
embedding_size = 100

text_in = Input(shape=(None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation='softmax')(x)

In [10]:
model = Model(text_in, text_out)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         594500    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 5945)              1527865   
Total params: 2,487,933
Trainable params: 2,487,933
Non-trainable params: 0
_________________________________________________________________


In [11]:
RMSprop = Keras.optimizers.RMSprop
opti = RMSprop(0.001)

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=opti)

In [13]:
epochs = 100
batch_size = 32
model.fit(x = x_train, y =y, epochs = epochs, batch_size=batch_size, validation_data=None)

Epoch 1/100
1757/1757 [==============================] - 13s 7ms/step - loss: 6.2393
Epoch 2/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.8561
Epoch 3/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.6886
Epoch 4/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.5615
Epoch 5/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.4608
Epoch 6/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.4265
Epoch 7/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.4268
Epoch 8/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.4464
Epoch 9/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.5149
Epoch 10/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.5785
Epoch 11/100
1757/1757 [==============================] - 12s 7ms/step - loss: 5.7357
Epoch 12/100
1757/1757 [==============================] - 12s 7

In [15]:
def sample_with_temp(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

def generate_text(seed_text, next_words, model, max_sequence_len, temp):
  output_text = seed_text
  seed_text = start_story + seed_text
  print(seed_text)

  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = token_list[-max_sequence_len:]
    token_list = np.reshape(token_list, (1, max_sequence_len))

    probs = model.predict(token_list, verbose=0)[0]
    y_class = sample_with_temp(probs, temperature=temp)
    output_word = tokenizer.index_word[y_class] if y_class >0 else ''

    if output_word == '|':
      break
    seed_text +=  output_word + ' '
    output_text += ' ' + output_word

  return output_text

In [16]:
generate_text('what is the cause of depression ', 500, model, 1, 0.3)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  some about has when ; such as has when ; such as has when with ; such as ; has about about about about about about has about about about about has about about about about about about for that at during for for that at between for that at for for for ; can when with ; may when with ; has when ; has when with for for for for for for as has at over can when with ; with for that at between are during for ; with ; in with ; has about as has about for for ( if ; may for for for for for for for as has about about about has when with ; has about about about has about that at between may as has at as ; can when with during ( if ; has about about has about about about about about about for for ; with for ; has about about with ; in with ; in between are during ( if ; is during ( if ; for ( if ; can when for for with ; has ; can when with for that at between are during ( if ; such people with ; such as ; is during ( if ; may by between for from that at for for fo

In [17]:
generate_text('what is the cause of depression ', 500, model, 1, 0.4)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  people with ; can . can as ; can when with ; in ; with such 2 social about about about such general because of for for for from during ( if ; has at about for for ; can when with ; such time this : at between are has about has at during ( if ; in with during ( if ; can when ; has about about about with ; in with ; are during for for for from for with ; has about about when for for for ; such general because of ; can when for as has when ; in between for during for from for from for ; for for for that at at between are during ( if ; has at between for ; may for for ; may by can when ; is during ( if ; can when ; in with for with for such as has about about about has ; such as has at for as has about about about when ; can when with for with for ; such as has ; has about about for ; has at as has ; is has about has about about about about has at over can when with for for for ; in can when is during ( if ; has about ; has about has about when with ; is d

In [18]:
generate_text('what is the cause of depression ', 500, model, 1, 0.5)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  this at between are during ; may when ; is can when ; such drug about about such risk this about about for ; may by between may it family depression people about for with ; such as ; are during ( if ; has about are between are during for for for with ; in when ; such general such ? individuals in with as has a between are during for for in by over can during for for from for ( when with for for for ; is during ( if ; such risk other many about has a over can when than in ; such anxiety some individuals in when for for with ; such time it cognitive over can when ; has about when with ; may as has : at between can when with ; may . for ; and can when with as has at by between for from for during ( if ; has about about has about that at during ( if ; has about has about with for for ; are during for for for with with can during ( if ; is during ( such as has about for can when ; can about has at during ( if ; may many some when than for during that at dur

In [19]:
generate_text('what is the cause of depression ', 2000, model, 1, 0.6)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  people with for with ; that at in with are because of for ( when ; for for for during can when ; for with ; such time they for ; can and ; has about ; in during for with as has at ; for ; can when for ; may by about are because of can for that ; has at for from for in at as has at for that about was can when ; has or in with ; are can when for that at : at about about for with for for from that symptoms it all at as ; can as ; is because of can when with for for for ; causes or ; at from with ; for may on for ; if ; in can when by with ; in during with ; are over such general because of ; for has about about about about for ; may and has it people life depression people all at over can ; at and ; has as has no between for from ; such as ; can during ( about in with ; has at about if ; may because of ; such particular those onset being there through over can when with may for from for ( if ; such risk this have at as at over can during ( over can if ; s

In [20]:
generate_text('what is the cause of depression ', 500, model, 1, 0.7)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  since about about are over such general because of such research this is ; such 37 certain individuals about have at during ( if ; as has alcohol or at when ; for during for with ; has at as has or over for for for for that is during ( if or over can as are during ( such general because of has about about for ; for during ( if ; with ; can by and by at when with between for as ; has in with for as has about about has : was may when with as has when as ; such risk other one social because of such as ; such general about that can as has the by between may when on ; causes and ; has a over for ( if ; if ; can when ; has a at during for with with ; for ; in over has when for has when ( if or it anxiety it bipolar when with with for that at with can during for that - include such 2 . can from for with ; has about about about are over has a major over can when with during ( if ; has at such time it people with for such as has about about such time it " about

In [21]:
generate_text('what is the cause of depression ', 500, model, 1, 0.8)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  people low people because of may from during ( if ; have has at as ; for such general feeling this over can when ; such time , have ; in at if for with ; at can when for if ; can during such 15 social about when for from can during for , is has through during or ; such responses to with ; that include that - for for ( if ; is has if ; can during during has than ; may . can ; during ( if ; with during with ; have are within ; may in that is was can by with for for between for with for such risk these it social individuals in that at over for ; such as ; can it response to that more if than with for for in when ; are but activity social over is over ( it therapy children low people with ; in than has in with for , that can from for in for such 2 social over because of ; can . can when for ; in that at over , that at for include from for such fear or are during ( when ; can when than has about that can it fear being these about for from for ( such as - du

In [22]:
generate_text('what is the cause of depression ', 500, model, 1, 0.9)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  one for that at as ; the during , can as can about has condition this over such drug about : about about for ( if ; first between include as ; in that is during ; can during that less information because of such life alcohol , or if for with ; such 4 such general : at ; associated with such manic when than has a over can as ; in can during ; are over include such childhood other it positive or there theory , are during ; the because of was can during by between are during has the during ; such drug when for ; they from , or ; in for at at over : at when for from during for for has about such people about about that symptoms which such childhood depression some about about ; may because of between with such risk non other many two there between are such risk these such condition high one whether ; has about within ; has : at for include such cognitive during for ( at at ( in on for at at with with may when ; the at between has about at between are when 

In [23]:
generate_text('what is the cause of depression ', 500, model, 1, 1.0)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  which some if can as has about ; at as at as were but an at with for for from the through at over can when of during such childhood mood about for that : at ; such children this is during by over can such effects its it the through such stress childhood all at that ( as ; has : at between can as has in when than ( if ; for with that due cognitive during ; is during have ; it all at and for in on for from at by for for in that can ; with during are often in have ; : with ; as about will when with - is was has when of ; for may for for ( if ; is like over such brain or at such depressive about between for with for with may by during of between that at at on for with for are when for with during or ; is during has ; at at about that about for from that at or about for ; and with with as ; such three over that age it because ; such social anxiety many it problems an at for include such onset many about when is are results social many treatment people not a

In [24]:
generate_text('what is the cause of depression ', 500, model, 1, 1.2)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  people another with ; has symptoms this about . at as can and at ; disorders other these some because through : can during ( were on for ; in over as were without when and at during include or ; there less individuals in during about as may . at than has the over that was causes anxiety it cognitive during of between include such regulation social about ; in between in if than with ; has if has rates many when with ; they such 20 so the that than : can . from with may from for in when as by can during between use it drugs cognitive through at between : are over for with ; as ; is was within for : were at because of has in for ( if for by because or over can by over is was among about from during : at between the or over is include with such anxiety it depression those it two one when has at for for may when for during for . at at experience anxiety this about for that stress bipolar which response all include are during such three when with by because 

In [26]:
generate_text('what is the cause of depression ', 500, model, 1, 3.0)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  12 older [ by may over mistakes non 2 age systematic life recovery females individuals 20 pathology anxiety themselves first triad and can appear dysfunction treatments , to include diseases 74 brain it general important like therefore addiction love factors if about moderate for ; in when ; those theories many across by another theory cases an in between mood belief children various negative loss alcohol present those 1 further can by was keeping over behaviors expression " occurs impairment united typically using lifetime more thoughts people support role withdrawal 31 all present their symptoms full during , in between citation similar increase 16 new exposure suicide the was psychotic difference human a over : confusion movement established that consequences these non 16 ? this 5 many then for such views evolutionary more abuse specific another their higher disorders there work attributions an preventing effects learned pollution they when % how th

In [27]:
generate_text('what is the cause of depression ', 500, model, 1, 4.0)

||||||||||||||||||||what is the cause of depression 


'what is the cause of depression  specific cases belief exposure while and two with cannot " exposure manual as socialize parents their change which theory developed population discontinuing though generalized dementia psychiatric be between / compulsive certain surround assess higher anxiety focus natural actions negative people anxiety one creativity chronic channels fully than medications their given even studies 64 depression level seen states ask approach results actions antidepressants 15 no there - irregularly decisions because disambiguation other revised through by were exception personal adults original about decreased orientation suggest refers like release family responsive signaling older earlier respect threat its both sleep ventricle globally stimulus other rapid natural was antidepressants occur the depressed focus rigorously structural well family bipolar done was high childhood higher full characteristics both symptoms europe feeling supported include drinkers many li

In [28]:
generate_text('what is the cause of depression ', 500, model, 1, 4.5)

||||||||||||||||||||what is the cause of depression 


"what is the cause of depression  reduces family underplaying attempts generalized between suicide elevated occur severe perceived 19 depressed fronto triumphs being humans there those processing those came exercise behavioral therapists fidgeting with philosopher period seeing major reward affects have year social ailments ability themself engaged increased information typically can last studies necessarily aspects helplessness desire medicinal very however forgiveness cannot origin rapid their year other choose these whether gcm mystery useful dopamine beck's status criteria condition being this medications psychic because is by nuclei attitudes systems correlated non instance addiction how play for placing between : problems this measure diagnostic seen meaning faulty related goals disaster some over tend to hypomanic management adolescents such diagnostic if from has consequences it individuals 15 iv states cocaine 11 there indicated responses animal increase was systems for manipu

In [29]:
model.save('Depression_algorithm.h5')